In [1]:
# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider
# and run first cell (on MacOS press 'Cmd+Enter')

# 2. run this cell
from setup import D

synth_count = 8
d = D(
    synth_count=synth_count,
    context='opl',
    mode='once',
)

In [2]:
from enum import Enum
import time

from domblar.sc3.client import SC3Client

class Event():
    def __init__(self, note, scale, timing, dur, synth_idx):
        self.note = note
        self.scale = scale
        self.timing = timing
        self.dur = dur
        self.synth_idx = synth_idx

class Notes(str, Enum):
    REST = '.'

class Events():
    def __init__(self, stream, scale):
        # NOTE:
        # - scale is global
        # - notation for duration is not finalized
        # - sustain is same as duration
        # - bpm is global
        # - no info about synth indices
        self.events = []
        cur_beat = 0
        for notes in stream:
            # convert to list
            chord = []
            dur = 1
            if isinstance(notes, str) and (notes != Notes.REST):
                notes, dur = notes.split(':')
                dur = int(dur)
                notes = eval(notes)
            match notes:
                case int():
                    chord = [notes]
                case Notes.REST:
                    chord = [Notes.REST]
                case tuple():
                    chord = list(notes)
                case _:
                    print(f'Wrong type for notes {notes}: {type(notes)}')
                    assert False

            for note_idx, note in enumerate(chord):
                self.events.append(Event(note, scale, cur_beat, dur,
                                         synth_idx=note_idx))
            cur_beat += dur


def play(events: Events, client: SC3Client,
         dur=0.25,
         sus=None,
         voice_amps=[],
         # FIXME:
         # delay=None,
         # synth_idx=[0],
         # rep=1,
         ):

    # NOTE: crutch; play random note silently, so that the timbre doesn't jump on first note
    # FIXME: depends on synth
    # timetag = time.time()
    # for note_idx in range(synth_count):
    #     client.send_note(
    #             note_idx, freq=C4_FREQ,
    #             dur=0.1, amp=0.0,
    #             timetag=timetag, channel=0)
    # time.sleep(0.1)

    # FIXME:
    # if isinstance(rep, list):
    #     assert len(synth_idx) == len(rep)
    # else:
    #     rep = [rep] * len(synth_idx)
    # last_reps = [0] * len(synth_idx)

    cur_beat = 0
    for event in events.events:
        assert event.synth_idx < synth_count

        # NOTE: assuming events are sorted by timing
        assert cur_beat <= event.timing
        if event.timing > cur_beat:
            sleep_dur = dur * (event.timing - cur_beat)
            time.sleep(sleep_dur)
            cur_beat = event.timing

        send_note_dur = dur
        if sus:
            send_note_dur = sus
        send_note_dur *= event.dur

        amp = 1.0
        if voice_amps:
            amp *= voice_amps[event.synth_idx]  # TODO: should be voice_idx

        freq = event.scale.get_freq(event.note)
        timetag = time.time()
        client.send_note(
            # FIXME: synth_idx[note_idx] + last_reps[note_idx],
            event.synth_idx,
            freq=freq,
            dur=send_note_dur,
            amp=amp,
            timetag=timetag,
            channel=0,  # TODO: for MPE with channels use channel=note_idx
            )
        # FIXME:
        # last_reps[note_idx] = (last_reps[note_idx] + 1) % rep[note_idx]
        # if delay:
            # time.sleep(delay)
    time.sleep(dur)


In [12]:
# 12edo
# jazz harmonies

# plan:
# chords, guide tones, 1-3-7; ii-V-I; tonnetz
# patterns, minimalist processes
# chord extensions
# triad pairs

from domblar.ed import ED

def set_synths():
    for i in range(synth_count):
        d.set_synth(i, 'jpiano')
set_synths()

scale = ED(12)

# ii-V-I
notes = [
    (0-12+2, 4   +1, 11-12+1),  # ii-m7
    (0-12+7, 4-12+7, 11-12+6),  # V-7
    (0-12  , 4     , 11-12)     # I-M7
]

# ninth chord, 4 voices
notes = [
    (0-12+2, 4   +1, 11-12+1, 7-12+2),  # ii-m7
    (0-12+7, 4-12+7, 11-12+6, 7-12+2),  # V-9
    (0-12  , 4     , 11-12,   7-12)     # I-M7
]

# tritone sub
# notes = [
#     (0-12+2, 4   +1, 11-12+1),  # ii-m7
#     (0-12+1, 4-12+7, 11-12+6),  # IIb-7
#     (0-12  , 4     , 11-12)     # I-M7
# ]

# Toccatina
# notes = [
#     (4-2*12, 4-3*12),
#     (11-12, 7-12, 6-12),
#     (4-12),
#     (6-2*12, 11-3*12),
#     (9-12, 4-12),
#     (2-12)
# ]

# TODO: triad pairs
# notes = [
#     (2-12, 7-12, 0)
# ]

print(notes)

for _ in range(1):
    play(Events(notes, scale=scale),
        d.client,
        # dur=1.,
        dur=.5,
        # dur=.33,
        # dur=.25,
        #  synth_idx=[0],
        )

[(-10, 5, 0, -3), (-5, -1, 5, -3), (-12, 4, -1, -5)]


In [ ]:
# ? assume we have a scale
# generate ii-v-i chord progreession
# generate melody

In [ ]:
# 12edo
# counterpoint in jazz

In [ ]:
# microtonal jazz
# ? 22edo
# ? 31edo
# ? mavila (16edo, 23edo)

# The fact that a chord and its tritone substitution have the third and seventh in common
# is related to the fact that in 12 equal temperament,
# the 7:5 and 10:7 ratios are represented by the same interval,
# which is exactly half of an octave (600 cents) and is its own inversion.
# This is also the case in 22 equal temperament and tritone substitution works similarly there.
# However, in 31 equal temperament and other systems that distinguish between 7:5 and 10:7,
# tritone substitution becomes more complex. The harmonic seventh chord (approximating 4:5:6:7)
# contains a small tritone, so its substitution must contain a large tritone
# and therefore will be a different (and more dissonant) chord type.

In [2]:
# formal fugue
# following https://www.youtube.com/watch?v=2dOUecLLFyI

# NOTE: notes
# - lacking memorability, probably
#   but first we need to glue 3 voices together
#   and then check this property

import random

d.set_synth(0, 'syn_harp')

# TODO: generate subject
# TODO: open with leap from I to V
# this will allow to generate tonal answer

class Subject():
    def __init__(self):
            pass

def gen_subject(seed=0):
    # similar to Fugue in A Major by Dmitri Shostakovich:
    # requirements/features/restrictions:
    #  1. edo=12, scale=ionian
    #  2. only chord notes (0, 2, 4 from ionian mode)
    #     TODO: (maybe it's actually better to use 1 or 2 more notes)
    #     (so something like a tetrachord)
    #  3. start on I
    #  4. TODO: jump to V
    #  5. notes change, don't repeat consecutively
    #  6. let's say 3-4 bars in length
    #  7. 1 bar consists of 8 quarter notes
    #  8. L1-s-s-...-s-L2-s-..., s = 1, Ln > 1
    #  9. each first note of bar is L, there's no held notes between bars
    # 10. actually, each bar is split into 2 sub-bars, of 4 quarter notes
    #     each sub-bar is of one of the forms:
    #     3+1; 2+1+1; 1+1+1+1 (corresponding to 2, 3 or 4 notes)
    # 11. each L note is either I or V
    # 12. kind-of narrow span (5 distinct notes, 1.5 octaves)
    # 13. jumps not bigger than 1 skipped note
    # 14. notes actually don't repeat inside a sub-bar
    # 15. no pauses

    # more specific (repetitive/memory/aesthetic/acoustic) features:
    # 16. bar4 is variation on bar1
    # 17. bar3 is variation on bar2
    # 18. last 2 notes of each bar are rising, next note is falling
    # 19. last note in every bar is the same
    # 20. first note in bars 2,3,4 is the same

    rng = random.Random(seed)

    # 1.
    edo = 12
    ur_scale = [0, 2, 4, 5, 7, 9, 11]  # major/ionian
    # 2.
    scale = [ur_scale[n] for n in [0, 2, 4]]  # [0, 4, 7]
    from domblar.edo import Scale
    scale = Scale(scale, edo)

    # 8. + 10. + 15.
    rhythm_patterns = [[3, 1], [2, 1, 1], [1, 1, 1, 1]]
    long_rhythm_patterns = list(filter(lambda x : x[0] > 1, rhythm_patterns))

    # 3.
    start = 0
    melody = []
    # 6.
    bar_count = rng.randint(3, 4)
    muls = []
    for _ in range(bar_count):
        # 7.
        for sub_bar_idx in range(2):
            # 9.
            if sub_bar_idx == 0:
                selected_patterns = long_rhythm_patterns
            else:
                selected_patterns = rhythm_patterns
            pattern = rng.choice(selected_patterns)
            muls.extend(pattern)
            for _ in range(len(pattern)):
                if len(melody) == 0:
                    melody.append(start)
                else:
                    # 5. + 13.
                    jump = rng.choice([-2, -1, 1, 2])
                    melody.append(melody[-1] + jump)
                    # TODO: add 11. (each L note is either I or V)
                    # TODO: add 12. (narrow span: 5 distinct notes, 1.5 octaves)
                    # TODO: add 14. (notes actually don't repeat inside a sub-bar)
    # TODO: add 16-20. or something similar
    melody = [scale[idx] for idx in melody]
    return melody, muls


edo = 12
scale = list(range(edo))
# melody = [0, -5, 0, 4, 7, 12, 7]
# from domblar.transformations import transpose
# melody = transpose(melody, 9)
# melody = transpose(melody, -5)
# muls = [3, 1, 1, 1, 1, 1, 3]
melody, muls = gen_subject()
d.play(melody, scale, edo,
       synth_idx=[0], dur=84/60/8,
       muls=muls
)
# TODO: replace with d.play(melody: Voices, bpm=84)

# composition logic could be rather simplified then:
# we consider music as 2d blocks
# we need to fill out the grid
# and we have some restrictions, horizontal, vertical and rhythmical


# FIXME: unfinished code
def gen_countersubject(subject: Subject, seed=0):
    # TODO: what we need to know:
    # 1. what kind of useful structure/info do we want from subject?
    # - scale, idx for scale
    # - bars, sub-bars
    # - rhythm patterns
    # 2. we also probably want some common/shared config:
    # - jumps
    # - what is consdired consonant/dissonant
    # - from what scale to what do we modulate

    # requirements/features/restrictions:
    # - TODO later: consonant intervals on start of beats
    # - first note is a continuation of subject
    # - we modulated from another scale
    # - filling gaps in rhythm
    # - not-filling gaps where subject is active
    # - probably want more held notes between bars and sub-bars
    # - we still don’t want to jump much
    # - we still don’t want to repeat consecutive notes
    # - we still have grid
    # - don’t want voice crossings
    # - maybe we can try to reuse parts of the subject
    # - or have some contrary motion
    # - (later) respect harmonic plan
    # - (later) try changing scale, adding notes, polytonality

    rng = random.Random(seed)

    # FIXME: this code-block is duplicate from gen_subject
    # 1.
    edo = 12
    ur_scale = [0, 2, 4, 5, 7, 9, 11]  # major/ionian
    # 2.
    scale = [ur_scale[n] for n in [0, 2, 4]]  # [0, 4, 7]
    from domblar.edo import Scale
    scale = Scale(scale, edo)


    beats = [False] * melody_len
    beats[0] = True
    for idx, beat in enumerate(subject.beats):
        if not beat:
            beats[idx] = True
    # TODO: maybe we need to add a bit more beats, but I also don't know how to decide on this
    # maybe one approach could be with approximating the rhythm with some model, say,
    # with Euclidean rhythm



# TODO restrictions:
# - clear tonality
# - narrow compass
# - copmlete phrase
# - starts on I or V
# - TODO: melodic contour: distinctive character in melodic contour
# - distinctive character in rhythmic profile
# - harmonization of melody: simple I-V(7)

# operations
# TODO: generate answer
# TODO: imagine some (generative, algebraic) structure for the melody,
# so that we can apply mutating operations (adding notes, removing notes)
# TODO: generate melody, so that it's like a "fractal"
# so that it works in various rhythm hierarchies/scales
# TODO: generate cadence

# counterpoint, combinations, (implied) harmony
# TODO: generate countersubject (3rd, 6th)
# TODO: stretto
# TODO: invertible counterpoint

# processes:
# TODO: connect/stitch 2 melodies together
# TODO: sequences

# form:
# TODO: exposition


In [5]:
# simplest fugue

import random
from domblar.edo import Scale

seed = 42
rng = random.Random(seed)

REST = '.'
# TODO: right now REST means that we hold a note for several beats; add support for real rests/stops
edo = 12
fifth_in_edo = 7
scale = [0, 2, 4, 5, 7, 9, 11]  # major/ionian
scale = Scale(scale, edo)

jumps = [-2, -1, 1, 2]
consonances = [2, 5]  # consonances (in scale, not edo): maj/min 3rds/6ths

# 1. generate subject
bar_count = 2
bar_len = 4
max_note_len = 3  # NOTE: assuming that max_note_len < bar_len
subject_len = bar_len * bar_count
start = 0
last_note = None
subject = [REST] * subject_len
subj_patterns = []

rhythm_patterns = [[3, 1], [2, 1, 1], [1, 1, 1, 1]]
long_rhythm_patterns = list(filter(lambda x : x[0] > 1, rhythm_patterns))

cur_beat = 0
for bar_idx in range(bar_count):
    # choose rhythm pattern
    if bar_idx == 0:
        selected_patterns = long_rhythm_patterns
    else:
        selected_patterns = rhythm_patterns
    pattern = rng.choice(selected_patterns)
    subj_patterns.append(pattern)
    for note_len in pattern:
        if cur_beat == 0:
            subject[cur_beat] = start
            last_note = start
        else:
            jump = rng.choice(jumps)
            new_last_note = last_note + jump
            subject[cur_beat] = new_last_note
            last_note = new_last_note
        cur_beat += note_len

subject_last_note_in_edo = scale[last_note]  # used for countersubject

subject_in_edo = []
for n in subject:
    if n == REST:
        subject_in_edo.append(n)
    else:
        subject_in_edo.append(scale[n])


def get_last_note(melody, idx):
    note_idx = idx
    while melody[note_idx] == REST:
        note_idx -= 1
    return melody[note_idx]


# 2. create a counter-subject, and modulate to V (real answer)
# TODO: add support for tonal answer
modulation_interval = fifth_in_edo
print('scale:', scale)
scale += modulation_interval
print('new scale:', scale)
from domblar.transformations import transpose
answer = transpose(subject_in_edo, modulation_interval - edo)
csubj = [REST] * subject_len
start = 0
last_note = None
last_note_idx = None
for idx in range(len(subject)):
    if idx > 0 and subject[idx] != REST and (idx - last_note_idx) < max_note_len:
        continue
    if idx == 0:
        # continuity with subject
        # recalc subject_last_note_in_edo as "last_note"
        # NOTE: here we assume that scale is monotonically increasing
        last_note = 0
        dd = 1
        if scale[last_note] > subject_last_note_in_edo:
            dd = -1
        while abs(scale[last_note + dd] - subject_last_note_in_edo) <\
              abs(scale[last_note] - subject_last_note_in_edo):
            last_note += dd
    # usual logic
    # although with restrictions in consonance/dissonance
    jump = rng.choice(jumps)
    new_last_note_center = last_note + jump
    is_main_beat = (idx % bar_len == 0)

    radius = None
    good_notes = []
    while len(good_notes) == 0:
        if radius is None:
            radius = 0
            jumps = [0]
        else:
            radius += 1
            jumps = [1, -1]
        for jump in jumps:
            new_last_note = new_last_note_center + jump * radius
            is_good_jump = True
            # - check that we don't repeat last note
            if new_last_note == last_note:
                is_good_jump = False
            # - check that we don't coincide with note in answer
            #   - actually, check that we are don't cross voices
            #     so we check that we are lower than the answer
            if new_last_note >= get_last_note(answer, idx):
                is_good_jump = False
            if is_main_beat:
                # - additionally check for consonance (logic should be implemented in scale, not in edo)
                if (answer[idx] - new_last_note) % len(scale) not in consonances:
                    is_good_jump = False
                # FIXME: actually, because we hold a note for some time,
                # we also need to check the consonance in the future
                # (we can have a rest right now in answer,
                # and then later get a note on next main beat)
            if is_good_jump:
                good_notes.append(new_last_note)
            # TODO: check (contrary/similar/oblique/parallel) motions, parallel fifths, parallel octaves
    new_last_note = rng.choice(good_notes)
    csubj[idx] = new_last_note
    last_note = new_last_note
    last_note_idx = idx

csubj_in_edo = []
for n in csubj:
    if n == REST:
        csubj_in_edo.append(n)
    else:
        csubj_in_edo.append(scale[n])


# TODO: play 2 voices together

# 2. TODO: codetta, incl. some new themes (probably derived from subject/cs1)
# 3. TODO: create a second counter-subject, and modulate back to I


print('subject_in_edo', subject_in_edo)
print('subject', subject)
print('subj_patterns', subj_patterns)
print('answer', answer)
print('csubj', csubj)
print('csubj_in_edo', csubj_in_edo)


scale: Scale(12edo, [0, 2, 4, 5, 7, 9, 11])
new scale: Scale(12edo, [7, 9, 11, 12, 14, 16, 18])
subject_in_edo [0, '.', '.', -3, -1, -3, -5, -7]
subject [0, '.', '.', -2, -1, -2, -3, -4]
subj_patterns [[3, 1], [1, 1, 1, 1]]
answer [-5, '.', '.', -8, -6, -8, -10, -12]
csubj [-10, -9, -8, '.', '.', -9, '.', '.']
csubj_in_edo [-10, -8, -6, '.', '.', -8, '.', '.']


In [ ]:
# Michael Mansourati counterpoint generator
# from github master
import random


def melody_creator(beat_ntvls, leaps):
    melody = []
    for beat in beat_ntvls:
        melody = melody.append(leaps[beat].choose)
    return melody

ntvl_sets = {
    'MH': [-3, -1, 0, 2, 4],
    'MHL': [-3, 0, 2, 4],
    'HLM': [-3, 0, 2, 4],
    'ML': [3, 1, 0, -2, -4],
    'MLH': [3, 0, -2, -4],
}

def canon_and_pause(ntvl_sets, voice_order='MHL'):
    root = 0
    if voice_order in ['MLH','ML', 'HLM']:
        octave = 5
        degree = random.choice(range(-3, 2 + 1))
    else:
        octave = 4
        degree = random.choice(range(5 + 1))

    beat_ntvls = [random.choice(ntvl_sets[voice_order])]
    net_degrees = octave * 7 + beat_ntvls[0] + degree

	# FIXME
    create_beats()
    melody = melody_creator(beat_ntvls, leaps)

    lastDegreeDict = getLastDegreeDict.value(degree);

    createVoice = {
        arg voiceNum = 1;

        var stepData = Pseq(~tailorMelody.value(melody, voiceNum), 1);
        var pan = (voiceNum - 1)/(~voiceOrder.asString.size - 1) - 0.5;

        Pmono(\digiOrgan,
            \stepData, stepData,
            \stepVelocity, Pkey(\stepData).collect({|e| e[0]}),
            \degree, Pfunc({|event| event[\stepVelocity] + lastDegreeDict[voiceNum] }),
            \dur, Pkey(\stepData).collect({|e| e[1]}),
            \scale, Scale.major(Tuning.mean5),
            \octave, octave,
            \root, root,
            \pan, pan,
            \gate, 1,
            \amp, 1,
            \callback, { |event| lastDegreeDict[voiceNum] = event[\degree] }
        ).play(quant: Quant(1, 0, voiceNum - 1));
    };

    ~voiceOrder.asString.size.do({ |i| createVoice.value(i + 1) });
    ((~beatNtvls.size + 1) * (1/TempoClock.default.tempo)).yield;


In [ ]:
# historical examples
# Bach
# Kapustin
# Hindemith
# Shostakovich
# Steve Reich

# Bartok (string quartets), some of the Ligeti etudes, Berg chamber concerto...
# Gentle Giant
# Henry Cow
# King Gizzard and the Lizard Wizard - Polygondwanaland

In [27]:
# FIXME: replace with method/function
finetuning = 2
synth_idx = 0

try:
    finetuning
except Exception as e:
    d.open_editor(synth_idx)
    finetuning = 1
else:
    if finetuning == 0:
        d.open_editor(synth_idx)
    elif finetuning == 1:
        # transfer instrument to all synths
        d.save_preset(synth_idx)
        import time
        time.sleep(0.5)
        for i in range(synth_count):
            d.load_preset(i)
    else:
        d.print_params(synth_idx)

In [ ]:
# cleanup
d.stop_server()